In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [ ]:
# data_file = '/data/shared/satellite_derived_feature/MA/MA_features_resnet50_imagenet.feather'
# data_file = '/data/shared/satellite_derived_feature/MA/MA_features_resnet50_swav_imagenet.feather'
# data_file = '/data/shared/satellite_derived_feature/Denmark/Denmark_features_resnet50_imagenet.feather'
# data_file = '/data/shared/satellite_derived_feature/Denmark/Denmark_features_resnet50_swav_imagenet.feather'
# data_file = '/data/shared/satellite_derived_feature/MI/MI_features_resnet50_swav_imagenet.feather'
# data_file = '/data/shared/satellite_derived_feature/MI/MI_features_resnet50_imagenet.feather'
# data_file = '/data/shared/satellite_derived_feature/Mexico/Mexico_features_resnet50_imagenet.feather'
data_file = '/data/shared/satellite_derived_feature/Mexico/Mexico_features_resnet50_swav_imagenet.feather'
data = pd.read_feather(data_file)

In [ ]:
data

In [ ]:
feather_array = data[['features_{}'.format(i) for i in range(2048)]].values

In [5]:
# Start the clustering
from sklearn.cluster import KMeans
random_state = 42
cluster_list = [10, 20, 30, 40, 50, 100, 200]
cluster_dict = {}
for n_cluster in cluster_list:
    y_pred = KMeans(n_clusters=n_cluster, random_state=random_state).fit_predict(feather_array)
    cluster_dict[n_cluster] = np.copy(y_pred)



KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


In [ ]:
# Put the clustered index back into the data frame
for n_cluster in cluster_list:
    data['cluster_index_{}'.format(n_cluster)] = cluster_dict[n_cluster]

In [ ]:
# Save the data frame to a feather format
data.to_feather(data_file)

In [ ]:
data

In [ ]:

import math
def mean_white_balance(img):
    """
    第一种简单的求均值白平衡法
    :param img: cv2.imread读取的图片数据
    :return: 返回的白平衡结果图片数据
    """
    # 读取图像
    b, g, r = cv2.split(img)
    r_avg = cv2.mean(r)[0]
    g_avg = cv2.mean(g)[0]
    b_avg = cv2.mean(b)[0]
    # 求各个通道所占增益
    k = (r_avg + g_avg + b_avg) / 3
    kr = k / r_avg
    kg = k / g_avg
    kb = k / b_avg
    r = cv2.addWeighted(src1=r, alpha=kr, src2=0, beta=0, gamma=0)
    g = cv2.addWeighted(src1=g, alpha=kg, src2=0, beta=0, gamma=0)
    b = cv2.addWeighted(src1=b, alpha=kb, src2=0, beta=0, gamma=0)
    balance_img = cv2.merge([b, g, r])
    return balance_img
def increase_brightness(img, value=60):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img
    


In [ ]:
# Visualize the same cluster imgs

def visualize_images_batch(img_list ,n_by_n=3):
    """
    Visualize the images into a big batch
    """
    assert len(img_list) == n_by_n * n_by_n, 'Your number of img provides is {} but the plotting wants to do {} by {}'.format(len(img_list), n_by_n, n_by_n)
    f = plt.figure(figsize=[15,15])
    z = 1
    for i in range(n_by_n):
        for j in range(n_by_n):
            ax = plt.subplot(int('{}{}{}'.format(n_by_n, n_by_n, z)))
            img = mpimg.imread(img_list[z - 1])
            # Version 1 white balance
            # Version 2 white balance
            result = mean_white_balance(img)
            # image_norm = cv2.normalize(result, None, alpha=0,beta=200, norm_type=cv2.NORM_MINMAX)

            result = increase_brightness(result)
            imgplot = plt.imshow(result)
            plt.axis('off')
            z += 1
    plt.tight_layout()
    plt.show()
    

# Visualize the clusters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data_file = '/data/shared/satellite_derived_feature/MA/MA_features_resnet50_imagenet.feather'
data = pd.read_feather(data_file)

In [ ]:
visualize_images_batch(data['img'][:9])